In [1]:
import chess.pgn
import sys

f1 = open('white_wins.txt', 'w')
f2 = open('black_wins.txt', 'w')
pgn = open("train.pgn")
n_games = 0

while True:
    game = chess.pgn.read_game(pgn)
    try:
        result = game.headers["Result"]
    except:
        break

    if result == "1-0":
        n_games =+ 1
        board = game.board()
        for move in game.main_line():
            board.push(move)
            f1.write(str(board))
            f1.write("\n- - - - - - - -\n")
        f1.write("\n= = = = = = = =\n")
    if result == "0-1":
        n_games =+ 1
        board = game.board()
        for move in game.main_line():
            board.push(move)
            f2.write(str(board))
            f2.write("\n- - - - - - - -\n")
        f2.write("\n= = = = = = = =\n")

f1.close()        
f2.close()
pgn.close()
print("Done. ", n_games, " games parsed")

Done.  1  games parsed


In [1]:
import tensorflow as tf
white = open('white_wins.txt', 'r')
black = open('black_wins.txt', 'r')



C:\Users\Shashank\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [25]:
import numpy as np
i = 0
# boardframe = np.zeros((1,7,8,8))
# boardframe[0,1,:,:] = 1
# print(boardframe[0,1,:,:])
# print(boardframe)

tst = np.load("A00-139_first_10000.npz")
boards = tst['arr_0']
targets = tst['arr_1']
print(boards.shape)
print(boards[0,1,:,:])
print(targets.shape)

(844782, 6, 8, 8)
[[-1  0  0  0  0  0  0 -1]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 1  0  0  0  0  0  0  1]]
(844782, 6)


In [26]:
print(boards[0,:,])

[[[ 0  0  0  0  0  0  0  0]
  [-1 -1 -1 -1 -1 -1 -1 -1]
  [ 0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0]
  [ 1  1  1  1  1  1  1  1]
  [ 0  0  0  0  0  0  0  0]]

 [[-1  0  0  0  0  0  0 -1]
  [ 0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0]
  [ 1  0  0  0  0  0  0  1]]

 [[ 0 -1  0  0  0  0 -1  0]
  [ 0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0]
  [ 0  1  0  0  0  0  1  0]]

 [[ 0  0 -1  0  0 -1  0  0]
  [ 0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0]
  [ 0  0  1  0  0  1  0  0]]

 [[ 0  0  0 -1  0  0  0  0]
  [ 0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0]
  [ 0  0  0 